In [16]:

############Adjustments##############

#options
PATH = '/media/sf_SF/Stage2021/targetedQE/' 
OPTION = 'area' # area or label
PNG = '/media/sf_SF/Stage2021/Projects/MRM_feces/Lactic_acid/std/resized/'
## Adjustments

filename_Y_labels = 'total_y_matrix.txt'

########################



# load libraries
import pandas as pd
import os
import random
from PIL import Image
import numpy as np


#set paths
path_data_in = PATH + 'data/input/' + 'MachineLearning/'
path_data_out = PATH + 'data/output/' + 'MachineLearning/'
path_data_X = PNG
path_data_y = path_data_in + 'Yarrays/' #labels


filename = path_data_y + filename_Y_labels

y = pd.read_csv(filename, sep = "\t")

filenames_X_train = []
filenames_X_test = []

directory_list = os.listdir(path_data_X)

random.shuffle(directory_list)
os.chdir(path_data_X)

i = 0
for filename in directory_list:
    if ".png" in filename :
        if i % 3 == 0: 
            filenames_X_test.append(path_data_X + filename)
        else:
            filenames_X_train.append(path_data_X + filename)
        i = i + 1


def load_X_if_matched_in_y(filenames_list,y):
    all_images_as_array=[]
    label=[] 
    area = []  
    global ordered_filenames
    ordered_filenames=pd.DataFrame()
    
    for filename in filenames_list:
        filename_wopath = filename.split('resized/')[1]
        filename_wopath = filename_wopath.strip(".png")

        matching_y = y[y.SampleTargetedcombination==filename_wopath]
        if len(matching_y) == 1:
            label.append(matching_y.iloc[0,5]) 
            ar = matching_y.iloc[0,3]
            area.append(ar)
            
        
            img=Image.open(filename)
            np_array = np.asarray(img)

            l,b,c = np_array.shape    
            np_array = np_array.reshape(l*b*c,)   
            all_images_as_array.append(np_array)
            
            file_area = pd.DataFrame(np.array([[filename_wopath,ar]]))
            ordered_filenames = ordered_filenames.append(file_area, ignore_index=True)
            
            
        if len(matching_y) != 1:
            continue
        
    if OPTION == 'area':
        option = area
    else:
        option = label       
    
    
    return np.array(all_images_as_array), np.array(option)

def load_prediction(filenames_list):
    all_images_as_array=[]

    for filename in filenames_list:

        img=Image.open(filename)
        np_array = np.asarray(img)

        l,b,c = np_array.shape    
        np_array = np_array.reshape(l*b*c,)   
        all_images_as_array.append(np_array)

    
    return np.array(all_images_as_array)
    
prediction = load_prediction(filenames_prediction)
X_train,y_train = load_X_if_matched_in_y(filenames_X_train, y)
X_test, y_test = load_X_if_matched_in_y(filenames_X_test, y)

print(len(y_train))
print(len(X_train))
print(len(y_test))
print(len(X_test))
print(len(prediction))

print(ordered_filenames)

36
36
18
18
0
                             0                   1
0     200929s002_Dopamined4HCL  3351540725.7382803
1   200929s040_Lalanined3333D3    2072079.31864908
2     200929s015_Dopamined4HCL    602095705.052871
3       200929s044_LLacticacid    15612212500.9243
4   200929s013_Lalanined3333D3    80428.3561022166
5     200929s009_Dopamined4HCL    622301558.662578
6   200929s045_Lalanined3333D3  100526.91729363901
7     200929s039_Dopamined4HCL    565346437.426464
8   200929s002_Lalanined3333D3    49397.7532518717
9   200929s048_Lalanined3333D3                 0.0
10      200929s002_LLacticacid     605762757.71143
11    200929s043_Dopamined4HCL      587732573.1865
12    200929s041_Dopamined4HCL     586637284.62658
13  200929s015_Lalanined3333D3    199639.214280302
14    200929s016_Dopamined4HCL    592851657.218494
15    200929s048_Dopamined4HCL  3759889678.2855897
16      200929s012_LLacticacid    13701643881.0486
17      200929s009_LLacticacid    9101507557.84202


In [17]:
import pickle
loaded_model = pickle.load(open('test_saving_regressor.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

y_pred = loaded_model.predict(X_test)

0.7073469460017836


In [18]:
df = pd.DataFrame(data=y_pred)
print(df)

               0
0   2.583584e+09
1   1.167834e+08
2   9.147335e+08
3   1.593685e+10
4   3.404210e+05
5   6.036271e+08
6   9.965605e+05
7   4.543794e+08
8   2.586018e+05
9   1.774294e+08
10  1.139093e+10
11  8.430097e+08
12  5.684682e+08
13  2.001126e+05
14  8.988996e+08
15  2.813466e+09
16  1.389879e+10
17  9.605531e+09


In [19]:
result=pd.concat([ordered_filenames, df], axis=1)

In [20]:
print(result)

                             0                   1             0
0     200929s002_Dopamined4HCL  3351540725.7382803  2.583584e+09
1   200929s040_Lalanined3333D3    2072079.31864908  1.167834e+08
2     200929s015_Dopamined4HCL    602095705.052871  9.147335e+08
3       200929s044_LLacticacid    15612212500.9243  1.593685e+10
4   200929s013_Lalanined3333D3    80428.3561022166  3.404210e+05
5     200929s009_Dopamined4HCL    622301558.662578  6.036271e+08
6   200929s045_Lalanined3333D3  100526.91729363901  9.965605e+05
7     200929s039_Dopamined4HCL    565346437.426464  4.543794e+08
8   200929s002_Lalanined3333D3    49397.7532518717  2.586018e+05
9   200929s048_Lalanined3333D3                 0.0  1.774294e+08
10      200929s002_LLacticacid     605762757.71143  1.139093e+10
11    200929s043_Dopamined4HCL      587732573.1865  8.430097e+08
12    200929s041_Dopamined4HCL     586637284.62658  5.684682e+08
13  200929s015_Lalanined3333D3    199639.214280302  2.001126e+05
14    200929s016_Dopamine

In [11]:
result.to_csv('png_with_prediction2.txt', sep="\t")